In [9]:
from datasets import Dataset
import requests
import time
import numpy as np
import tensorflow as tf
import pandas as pd
import os


# API endpoint
url = "https://db.ygoprodeck.com/api/v7/cardinfo.php"

# Request parameters
params = {

}

word_list = dict()
subtype_list = dict()
char_list = dict()
card_number_listed = 0

try:
    response = requests.get(url, params=params)
    data = response.json()

    if response.status_code == 200:
        cards = pd.DataFrame(data=data["data"])
        
except requests.exceptions.RequestException as e:
    print("An error occurred:", e)

In [10]:
# Save the dataframe as csv
cards.to_csv("deck.csv", index=False)

In [12]:
import datasets
from PIL import Image

def load_image(row):
    feature = datasets.Image(decode=False)
    try:
        image = Image.open(row['image']).convert("RGB")
    except:
        print(f"Failed to open image {row['id']}")
        return

    encoded_example = feature.encode_example(image)
    if 'image' in encoded_example:
        return {'image': encoded_example['image']}
    else:
        print(f"Failed to encode image {row['id']}")
        return



df_c = cards[["id", "name"]].head(10)
df_c['image'] = df_c.apply(lambda row: f"images/{row['id']}.jpg", axis=1)

image_data = []
for _, row in df_c.iterrows():
    image_row = load_image(row)
    image_data.append(image_row)

df_c['image_data'] = image_data
df_c

Failed to encode image 34541863
Failed to encode image 64163367
Failed to encode image 91231901
Failed to encode image 73262676
Failed to encode image 98319530
Failed to encode image 37478723
Failed to encode image 64867422
Failed to encode image 90861137
Failed to encode image 44256816
Failed to encode image 86988864


,id,name,image,image_data
0,34541863,"""A"" Cell Breeding Device",images/34541863.jpg,None
1,64163367,"""A"" Cell Incubator",images/64163367.jpg,None
2,91231901,"""A"" Cell Recombination Device",images/91231901.jpg,None
3,73262676,"""A"" Cell Scatter Burst",images/73262676.jpg,None
4,98319530,"""Infernoble Arms - Almace""",images/98319530.jpg,None
5,37478723,"""Infernoble Arms - Durendal""",images/37478723.jpg,None
6,64867422,"""Infernoble Arms - Hauteclere""",images/64867422.jpg,None
7,90861137,"""Infernoble Arms - Joyeuse""",images/90861137.jpg,None
8,44256816,1st Movement Solo,images/44256816.jpg,None
9,86988864,3-Hump Lacooda,images/86988864.jpg,None


In [8]:
print(df_c['image_data'][0]['image'])

{'path': None, 'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02p\x00\x00\x02p\x08\x02\x00\x00\x00\x86q\xcb\xf4\x00\x01\x00\x00IDATx\x9c\xec\xfdY\xb3,9\x92&\x88}\xaa\x80\x99\xb9\xfbY\xee\x1e\x11YYKWwuO\x93M\xbe\x8d\x08\xff\x03\x7f\x07\x7f\x1d\xff\x04\x1f\xf8H\xa1\x0ce\x86\xc3\x19NuMWWVe\xc6v\x97s\xcf9\xeen\x06@\x95\x0f\n\xc0`\xe6\xee\xe7.\x11\x99\x95\x14\x19H\xe4M;\xe60\xac\n\xdd\xa0\x0b\xfd_\xff/\xffgU\x15\x02\x00Q\x00P\xb2\x7f\x9d=$(\x94\x99\x99\xbds\xce\xb1$fv\x8e\x88\x08\x00\x111\x81\x88:v\x00\x88\x95\x88\x18\xf6\x93\x02 \xab\xc3L\xa5\x00\x00P\xdf\xa8*\x9a:\x1d\xc0\xcc\xce\xb9Z\xd3~\xad\xdd\xa1)J`f\x1b<\x11\x81\xf3\x13\x80\x94\x14\x804\xd55\xf7\xeb\x89lhPUU%UUe\x01\x11q\xe9\x0e\x80\xd5\xa9\x1d\x01P"%\x00\x9c\x08\x89(BBJ\x02\x15\xf0</\xe5v\x9c\xaa\x9a\xa0\xaa\nV\xfb\x13\x00\x83\x88\xa8#\xee\x08\x9d\xc2\xe7\xa1\x81t\x1e\x83\x88`Y\xeco\xd7\xac\x80\xb5vZ\xda\xf7\x9a\xd6\xed<\xf1m^\xcfy\x9cP\xd5\xbc\x0eM;\xed\xca\x08\x9e\x1a\x83\xadp\xfb\'\x9d\xaf\x0e,\xab\xd5\xe1\xb1\xd6\xdf\x

In [5]:
from datasets import Dataset, load_dataset
from huggingface_hub import HfApi

dataset = Dataset.from_pandas(df_c)
dataset

# Upload to Huggingface
dataset.push_to_hub("FabioArdi/test", private=True)

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:02<00:00,  2.10s/it]
